In [9]:
import os
import pandas as pd

In [12]:
df = pd.read_csv(os.getenv('PREDICTION_CSV_PATH'))

In [13]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Rank,Champion Name,Tier,Win rate,Pick Rate,Ban Rate,Matches,championId,...,role,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerLevel,summonerName,teamId,teamPosition,win
0,0,0,1,105.0,4.0,53.87,12.0,1.7,19216,50,...,3.0,3,4,6,3,546,17645.0,100,0.0,1.0
1,1,1,1,105.0,4.0,53.87,12.0,1.7,19216,50,...,0.0,3,4,4,6,105,20869.0,100,0.0,0.0
2,2,2,1,105.0,4.0,53.87,12.0,1.7,19216,50,...,0.0,1,4,4,6,212,22970.0,100,0.0,0.0
3,3,3,1,105.0,4.0,53.87,12.0,1.7,19216,50,...,0.0,5,6,3,4,681,28039.0,100,0.0,1.0
4,4,4,1,105.0,4.0,53.87,12.0,1.7,19216,50,...,4.0,3,14,3,4,671,33854.0,200,4.0,0.0


In [15]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1'], inplace=True, axis=1)

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Check for missing values
missing_values = df.isnull().sum()

# Drop rows with missing values
df.dropna(inplace=True)

# Split data into features and target variable
X = df.drop('win', axis=1)
y = df['win']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

missing_values, X_train.shape, X_test.shape

(Rank               0
 Champion Name      0
 Tier               0
 Win rate           0
 Pick Rate          0
 Ban Rate           0
 Matches            0
 championId         0
 championName       0
 lane               0
 playerAugment1     0
 playerAugment2     0
 playerAugment3     0
 playerAugment4     0
 playerSubteamId    0
 role               0
 summoner1Casts     0
 summoner1Id        0
 summoner2Casts     0
 summoner2Id        0
 summonerLevel      0
 summonerName       0
 teamId             0
 teamPosition       0
 win                0
 dtype: int64,
 (201901, 24),
 (50476, 24))

In [21]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Initialize and train the XGBoost model
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
accuracy

C:\Programmieren\Leaguify_Website\venv\Lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


0.5775616134400507

In [27]:
import torch
from bayesian_torch.models.dnn_to_bnn import dnn_to_bnn, get_kl_loss
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Define a Bayesian neural network architecture with more complexity
class VeryLargeBayesianNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size1, hidden_size2):
        super(VeryLargeBayesianNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size2)
        self.fc4 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Convert a deterministic neural network to Bayesian
def convert_to_bnn(model):
    const_bnn_prior_parameters = {
        "prior_mu": 0.0,
        "prior_sigma": 1.0,
        "posterior_mu_init": 0.0,
        "posterior_rho_init": -3.0,
        "type": "Reparameterization",  # Flipout or Reparameterization
        "moped_enable": False,  # True to initialize mu/sigma from the pretrained dnn weights
        "moped_delta": 0.5,
    }
    
    dnn_to_bnn(model, const_bnn_prior_parameters)

# Define hyperparameters
input_size = X_train.shape[1]
output_size = 1  # Assuming you are predicting a binary outcome (win or not)
hidden_size1 = 512  # Increased size of the first hidden layer
hidden_size2 = 256  # Added a second hidden layer with more neurons
learning_rate = 0.001
num_epochs = 1000  # Increased number of training epochs

# Create the larger Bayesian neural network
very_large_bnn_model = VeryLargeBayesianNN(input_size, output_size, hidden_size1, hidden_size2)

# Convert the model to Bayesian
convert_to_bnn(very_large_bnn_model)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss
optimizer = optim.Adam(very_large_bnn_model.parameters(), lr=learning_rate)

# ... (previous code for preprocessing)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)  # Reshape to (number_of_samples, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).reshape(-1, 1)  # Reshape to (number_of_samples, 1)

# ... (rest of the code)

# Training loop with increased number of epochs
for epoch in range(num_epochs):
    # Forward pass
    outputs = very_large_bnn_model(X_train_tensor)
    
    # Compute the KL divergence loss
    kl_loss = get_kl_loss(very_large_bnn_model)
    
    # Compute the binary cross-entropy loss
    loss = criterion(outputs, y_train_tensor) + kl_loss / len(X_train_tensor)
    
    # Backpropagation and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print the loss for monitoring
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
very_large_bnn_model.eval()
with torch.no_grad():
    # Make predictions on the test set
    test_outputs = very_large_bnn_model(X_test_tensor)
    predicted_labels = (torch.sigmoid(test_outputs) >= 0.5).float()
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, predicted_labels)
    print(f'Accuracy on the test set: {accuracy:.4f}')

# You can now use the model to make predictions on new data

Epoch [10/1000], Loss: 0.9307
Epoch [20/1000], Loss: 0.8038
Epoch [30/1000], Loss: 0.7327
Epoch [40/1000], Loss: 0.7418
Epoch [50/1000], Loss: 0.7174
Epoch [60/1000], Loss: 0.7134
Epoch [70/1000], Loss: 0.7143
Epoch [80/1000], Loss: 0.7102
Epoch [90/1000], Loss: 0.7102
Epoch [100/1000], Loss: 0.7092
Epoch [110/1000], Loss: 0.7016
Epoch [120/1000], Loss: 0.7002
Epoch [130/1000], Loss: 0.7088
Epoch [140/1000], Loss: 0.7051
Epoch [150/1000], Loss: 0.6997
Epoch [160/1000], Loss: 0.7011
Epoch [170/1000], Loss: 0.6997
Epoch [180/1000], Loss: 0.6992
Epoch [190/1000], Loss: 0.7005
Epoch [200/1000], Loss: 0.6986
Epoch [210/1000], Loss: 0.6997
Epoch [220/1000], Loss: 0.6969
Epoch [230/1000], Loss: 0.6956
Epoch [240/1000], Loss: 0.7070
Epoch [250/1000], Loss: 0.6965
Epoch [260/1000], Loss: 0.6959
Epoch [270/1000], Loss: 0.6961
Epoch [280/1000], Loss: 0.6982
Epoch [290/1000], Loss: 0.6956
Epoch [300/1000], Loss: 0.6969
Epoch [310/1000], Loss: 0.6971
Epoch [320/1000], Loss: 0.6958
Epoch [330/1000],